In [1]:
import boto3
import pandas as pd
from io import BytesIO

s3 = boto3.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)


manifolddata


In [7]:
client = boto3.client('s3')
obj = client.get_object(Bucket='manifolddata', Key='week1.csv')
df = pd.read_csv(BytesIO(obj['Body'].read()))

/home/ec2-user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.head()

,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flows,Flags,Tos,label,attackType,attackID,attackDescription
0,2017-08-02 00:00:00.419,0.003,TCP,192.168.210.55,44870,192.168.100.11,445.0,2,174,1,.AP...,0,normal,---,---,---
1,2017-08-02 00:00:00.421,0.000,TCP,192.168.100.11,445,192.168.210.55,44870.0,1,108,1,.AP...,0,normal,---,---,---
2,2017-08-02 00:00:02.593,0.004,TCP,192.168.220.47,55101,192.168.100.11,445.0,2,174,1,.AP...,0,normal,---,---,---
3,2017-08-02 00:00:02.859,0.000,TCP,10000_34,443,192.168.210.54,59628.0,1,100,1,.AP...,0,normal,---,---,---
4,2017-08-02 00:00:02.594,0.000,TCP,192.168.100.11,445,192.168.220.47,55101.0,1,108,1,.AP...,0,normal,---,---,---


In [14]:
### Data exploration
df=df.iloc[:,[0,1,3,4,5,6,7,8]]
df.columns=['Date', 'Duration', 'Src_IP', 'Src_pt', 'Dst_IP', 'Dst_pt','Packets', 'Bytes']

In [17]:
#add an date column that is rounded to nearest hour, so we can use this as a timestep to see how frequently IP pairs occur in each timestep
df['Date']=pd.to_datetime(df['Date'], format="%Y-%m-%d %H:%M:%S.%f", errors = 'coerce')
df['date_hr']=pd.Series(df['Date']).dt.round("H")

In [18]:
df.head()

,Date,Duration,Src_IP,Src_pt,Dst_IP,Dst_pt,Packets,Bytes,date_hr
0,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02
1,2017-08-02 00:00:00.421,0.000,192.168.100.11,445,192.168.210.55,44870.0,1,108,2017-08-02
2,2017-08-02 00:00:02.593,0.004,192.168.220.47,55101,192.168.100.11,445.0,2,174,2017-08-02
3,2017-08-02 00:00:02.859,0.000,10000_34,443,192.168.210.54,59628.0,1,100,2017-08-02
4,2017-08-02 00:00:02.594,0.000,192.168.100.11,445,192.168.220.47,55101.0,1,108,2017-08-02


In [43]:
#create a pair column, which is a touple of the src and dst IP, sorted. 
#It does not matter which call came first, we simply want to know which pair occurs most frequently.

df['pairs']=list(zip(df.Src_IP, df.Dst_IP))
df['pairs']=df['pairs'].apply(sorted)
df['pairs2']=tuple(df['pairs'])

NameError: name 'touple' is not defined

In [46]:
df.head()

,Date,Duration,Src_IP,Src_pt,Dst_IP,Dst_pt,Packets,Bytes,date_hr,pairs,pairs2
0,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)"
1,2017-08-02 00:00:00.421,0.000,192.168.100.11,445,192.168.210.55,44870.0,1,108,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)"
2,2017-08-02 00:00:02.593,0.004,192.168.220.47,55101,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)"
3,2017-08-02 00:00:02.859,0.000,10000_34,443,192.168.210.54,59628.0,1,100,2017-08-02,"[10000_34, 192.168.210.54]","(10000_34, 192.168.210.54)"
4,2017-08-02 00:00:02.594,0.000,192.168.100.11,445,192.168.220.47,55101.0,1,108,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)"


Group the data by the hour timestep, count how many times each pair occurs in that hour

In [ ]:
hour_group=df.groupby(['date_hr', 'pairs2']).count()
hour_group=hour_group['index']
hour_group.reset_index()

hour_group=hour_group.to_frame()

In [ ]:
hour_group.head()

In [40]:
import numpy as np
data = pd.Series(np.random.randn(100), index=pd.date_range('01/01/2001', periods=100))
keys = lambda x: (x.year,x.month)  # <----
xx=data.groupby(keys).mean()

In [39]:
keys

<function __main__.<lambda>(x)>

In [42]:
xx

(2001, 1)    0.221195
(2001, 2)   -0.040622
(2001, 3)   -0.216308
(2001, 4)    0.219584
dtype: float64